# Data Cleaning

All of the data was downloaded to CSV files directly from the website, except for the inflation data.  That was obtained through the following code:

In [1]:
import pandas as pd
dat = pd.read_html("https://www.usinflationcalculator.com/inflation/historical-inflation-rates/")
df = pd.DataFrame(dat[0])
df = df.set_index("Year")
df.to_csv('data/inflation.csv', index=True)

## Cleaning DJIA Data

The DJIA data from the Federal Reserve (FED) was loaded into a DataFrame called dow1, while the data from the Wall Street Journal (WSJ) was loaded into dow2.

In [2]:
dow1 = pd.DataFrame(pd.read_csv('data/dow1.csv'))
dow2 = pd.DataFrame(pd.read_csv('data/dow2.csv'))

Unnecessary columns are dropped from the WSJ data:

In [3]:
dow2 = dow2.drop(columns = [' Open', ' High', ' Low'])

Dates up to 1954 are dropped from FED data, and the index is set and the column is renamed.  For the WSJ data, the data is in reverse order, so dates from the current day to 2019 are dropped, and the order is reversed.  The index is reset so the indices match the number of entries.

In [4]:
dow1 = dow1.drop(list(range(457)))
dow1 = dow1.set_index('DATE')
dow1.columns = ['Close']
dow2 = dow2.drop([12698])
dow2 = dow2.drop(list(range(339)))
dow2 = dow2.iloc[::-1]
dow2.index = list(range(len(dow2)))

In the WSJ data, the rows are dropped if they are not the first of the month.   Then, for both datasets, to just get quarterly data, the rows are dropped if they are not in the months of January, April, July, or October.

In [5]:
dow2['Date'] = pd.to_datetime(dow2['Date'])
toKeep = []
comp = dow2.iloc[0]['Date'].day
for i in range(len(dow2)):
    ind = dow2.iloc[i]['Date'].day
    if(ind <= comp):
        toKeep = toKeep + [i]
    comp = ind
toDrop = []
for i in range(len(dow2)):
    if i not in toKeep:
        toDrop = toDrop + [i]
dow2 = dow2.drop(toDrop)
dow2.index = list(range(len(dow2)))
dow1 = dow1.reset_index()
dow1['DATE'] = pd.to_datetime(dow1['DATE'])
months = [1, 4, 7, 10]
toDrop = []
for i in range(len(dow1)):
    if(dow1.iloc[i]['DATE'].month not in months):
        toDrop = toDrop + [i]
dow1 = dow1.drop(toDrop)
dow1.index = list(range(len(dow1)))
dow1.columns = ['Date', 'Close']
toDrop = []
for i in range(len(dow2)):
    if(dow2.iloc[i]['Date'].month not in months):
        toDrop = toDrop + [i]
dow2 = dow2.drop(toDrop)
dow2.index = list(range(len(dow2)))

The data for 1969 and 1970 is manually gathered from the third source.  The three DataFrames are then concatenated, and the index is set to be simply numbers.  This is because the WSJ data has dates that are sometime the second, third, or fourth, but the GDP data will have all dates starting with 1, so that will become the index of the overall DataFrame.

In [6]:
dow3 = pd.DataFrame(data= {'Date': ['1/1/1969', '4/1/1969', '7/1/1969', '10/1/1969', '1/1/1970', '4/1/1970', '7/1/1970', '10/1/1970'], 'Close': [946.05, 950.18, 815.47, 855.99, 744.06, 736.07, 734.12, 755.61]})
dow3['Date'] = pd.to_datetime(dow3['Date'])
dow1 = dow1.set_index('Date')
dow2 = dow2.set_index('Date')
dow3 = dow3.set_index('Date')
dow2.columns = ['Close']
dow = pd.concat([dow1, dow3, dow2])
dow = dow.reset_index()
dow = dow.drop([0, 1, 2, 3])
dow = dow.drop(columns = ['Date'])
dow.index = list(range(len(dow)))
dow.columns = ['Value of Dow']

In [7]:
dow

,Value of Dow
0,286.64
1,310.92
2,341.27
3,358.29
4,398.43
...,...
259,26651.21
260,23346.24
261,26258.42
262,26717.43


## Cleaning GDP Data

This simply consisted of dropping the extra years, as well as setting the index to correctly index the data.  The date was kept, as it will be used as the index for the final DataFrame.

In [8]:
gdp = pd.DataFrame(pd.read_csv('data/gdp.csv'))
gdp.columns = ['Date', 'GDP Growth']
gdp = gdp.drop(list(range(23)))
gdp['Date'] = pd.to_datetime(gdp['Date'])
gdp.index = range(len(gdp))
gdp = gdp.drop([0, 1, 2, 3, 268, 269, 270, 271, 272])
gdp.index = range(len(gdp))
gdp

,Date,GDP Growth
0,1954-01-01,-1.9
1,1954-04-01,0.4
2,1954-07-01,4.6
3,1954-10-01,8.1
4,1955-01-01,11.9
...,...,...
259,2018-10-01,1.3
260,2019-01-01,2.9
261,2019-04-01,1.5
262,2019-07-01,2.6


## Cleaning Inflation Data

This consisted of dropping the data for years outside of the range, as well as keeping the inflation data for the months of January, April, July, and October.  The index is once again simply a list to index the values.

In [9]:
inf = pd.DataFrame(pd.read_csv('data/inflation.csv'))
inf = inf.drop([106, 107])
inf = inf.drop(list(range(40)))
inf.index = list(range(len(inf)))
vals = []
for i in range(len(inf)):
    vals = vals + [inf.iloc[i]['Jan']] + [inf.iloc[i]['Apr']] + [inf.iloc[i]['Jul']] + [inf.iloc[i]['Oct']]
inf = pd.DataFrame(data = {'Inflation Rate': vals})
inf

,Inflation Rate
0,1.1
1,0.8
2,0.4
3,-0.7
4,-0.7
...,...
259,2.5
260,1.6
261,2.0
262,1.8


## Cleaning Unemployment Data

This consisted of dropping the data for the years outside of the range, as well as keeping data for the months of January, April, July, and October.  The index was also reset.

In [10]:
unem = pd.DataFrame(pd.read_csv('data/unemployment.csv'))
unem = unem.drop(list(range(72)))
unem = unem.drop(list(range(864, 879)))
unem.index = list(range(len(unem)))
toDrop = []
for i in range(len(unem)):
    if(i % 3):
        toDrop = toDrop + [i]
unem = unem.drop(toDrop)
unem.index = list(range(len(unem)))
unem = unem.drop(columns = ['Series ID', 'Year', 'Period', 'Label', '1-Month Net Change'])
unem.columns = ['Unemployment Rate']
unem

,Unemployment Rate
0,4.9
1,5.9
2,5.8
3,5.7
4,4.9
...,...
259,3.8
260,4.0
261,3.7
262,3.6


## Cleaning Interest Rate Data

This consisted of dropping the data for the years outside of the range, as well as keeping data for the months of January, April, July, and October.  The index was also reset.

In [11]:
inter = pd.DataFrame(pd.read_csv('data/interestRates.csv'))
inter = inter.drop(list(range(48)))
inter = inter.drop(list(range(840, 855)))
inter.index = list(range(len(inter)))
toDrop = []
for i in range(len(inter)):
    if(i % 3):
        toDrop = toDrop + [i]
inter = inter.drop(toDrop)
inter = inter.drop(columns = ['DATE'])
inter.columns = ['Interest Rate']
inter.index = list(range(len(inter)))
inter

,Interest Rate
0,2.00
1,1.63
2,1.50
3,1.50
4,1.50
...,...
259,2.75
260,3.00
261,3.00
262,3.00


## Cleaning Real Median Family Income Data

This was only yearly data, so there needs to be data filled in.  Even though the CSV records every date as the first of the year, according to the website, the value reflects the value towards the end of the year.  Therefore, the value from 1953 was collected to help fill the data.  The overall shape of the real median family income as seen on the website suggests that the behavior of this data is linear locally, so that is how the data will be filled in.  The following is a function that will calculate the missing data.

In [12]:
#Function takes yearly data and will fill in missing data using a linear model
#For year1 data and year2 data:
#July is average of the data of year1 and year2
#April is average of July and year1 data
#October is average of July and year2 data
import statistics
def fillToQuarterly(data):
    compl = []
    for i in range(len(data) - 1):
        jul = statistics.mean([data[i], data[i + 1]])
        apr = statistics.mean([data[i], jul])
        oc = statistics.mean([jul, data[i + 1]])
        compl = compl + [data[i], apr, jul, oc]
    return compl

This data also only goes from 1953 to 2019, so there is no need to delete any data, as the 1953 data is only used to help fill the data.

In [13]:
inc = pd.DataFrame(pd.read_csv('data/income.csv'))
dates = inc['DATE'][1:]
dates[67] = '1/1/2020'
inc['DATE'] = list(dates)
inc.columns = ['Date', 'Real Median Family Income']
inc = pd.DataFrame(data = {'Real Median Family Income': fillToQuarterly(inc['Real Median Family Income'])})
inc

,Real Median Family Income
0,35650
1,35415
2,35180
3,34945
4,34710
...,...
259,79904
260,80071
261,81556
262,83041


## Joining the DataFrames 

The DataFrames are joined, the index is set to be the date provided by the GDP DataFrame, and is saved to a CSV file called cleandata.csv.

In [14]:
cleandata = pd.concat([gdp, inc, inf, unem, inter, dow], axis=1)
cleandata = cleandata.set_index('Date')
cleandata

,GDP Growth,Real Median Family Income,Inflation Rate,Unemployment Rate,Interest Rate,Value of Dow
Date,,,,,,
1954-01-01,-1.9,35650,1.1,4.9,2.00,286.64
1954-04-01,0.4,35415,0.8,5.9,1.63,310.92
1954-07-01,4.6,35180,0.4,5.8,1.50,341.27
1954-10-01,8.1,34945,-0.7,5.7,1.50,358.29
1955-01-01,11.9,34710,-0.7,4.9,1.50,398.43
...,...,...,...,...,...,...
2018-10-01,1.3,79904,2.5,3.8,2.75,26651.21
2019-01-01,2.9,80071,1.6,4.0,3.00,23346.24
2019-04-01,1.5,81556,2.0,3.7,3.00,26258.42


In [15]:
cleandata.to_csv('data/cleandata.csv', index=True)